In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import psColor, bwLabel
import os

TRAIN_DIR = os.listdir(path='../data/treino')
DATA = [cv2.imread('../data/treino/' + image_path) for image_path in TRAIN_DIR]
TARGETS = [7, 5, 3, 4, 2, 4, 0, 3, 4, 4, 4, 1, 4, 3]

In [ ]:
#1. Leitura de imagens

image= DATA[8]

cv2.imshow('Imagem Original',image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
#2. Binarização 

#Tirar background
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
blue_low= np.array([100, 100, 0])
blue_high= np.array([140, 255, 255])
background_mask= cv2.inRange(image_hsv, blue_low, blue_high)
inverted_mask= 255 - background_mask
image_objects= cv2.bitwise_and(image, image, mask=inverted_mask)

# cv2.imshow('Mask', inverted_mask)
cv2.imshow('Imagem sem backgound', image_objects)

image_gs= cv2.cvtColor(image_objects, cv2.COLOR_BGR2GRAY)

thresh, tresh_img= cv2.threshold(image_gs, 150, 255, cv2.THRESH_BINARY)

aux_1= cv2.cvtColor(tresh_img, cv2.COLOR_GRAY2BGR)
cv2.imshow('Imagem Binarizada', aux_1)

cv2.waitKey(0)
cv2.destroyAllWindows()